In [175]:
import pandas as pd # library for data analsysis
import requests
from bs4 import BeautifulSoup
print('Libraries imported.')

Libraries imported.


In [178]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())
My_table = soup.find('table',{'class':'wikitable sortable'})
#print (My_table)

table_classes = {"class": ["sortable", "plainrowheaders"]}
wikitb = soup.findAll("table", table_classes)
#wikitb

In [183]:
# define the dataframe columns
column_names = ['PostCode', 'Borough', 'Neighbourhood'] 
# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

column1 = []
column2 = []
column3 = []

for row in wikitb[0].findAll('tr'):
    cells = row.findAll(['th','td'])
    #print(cells)
    for c in cells[0].findAll(text=True):
        #print (c)
        column1.append(c)
    for c in cells[1].findAll(text=True):
        #print (c)
        column2.append(c)
    #print (cells[2])
    for c in cells[2].findAll(text=True):
        #print (c.strip())
        column3.append(c.strip())
   
column1.pop(0)
column2.pop(0)
column3.pop(0)

neighborhoods['PostCode'] = column1
neighborhoods['Borough'] = column2
#print (len(column1))
#print (len(column2))
#print (len(column3))

column3= [x for x in column3 if x!='']
#neighborhoods['Borough'] = column2
neighborhoods['Neighbourhood'] = column3
#neighborhoods

neighborhoods = neighborhoods[neighborhoods["Borough"]!="Not assigned"]
neighborhoods = neighborhoods.groupby(['PostCode','Borough'], as_index = False)['Neighbourhood'].agg(list)

#neighborhoods['Neighbourhood'] = neighborhoods['Neighbourhood'].map(lambda x: x.lstrip('[').rstrip(']'))
neighborhoods['Neighbourhood'] = neighborhoods["Neighbourhood"].apply(lambda x: ', '.join(x))

line = neighborhoods[neighborhoods["Neighbourhood"]=="Not assigned"]
line['Neighbourhood'] = line['Borough']
neighborhoods[neighborhoods["Neighbourhood"]=="Not assigned"] = line

neighborhoods

neighborhoods.shape

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(103, 3)

Assumption: there was only 1 table

Assumption: assuming the Columns' names

Assumption: all original columns had the same size